# generate NER data and bootstrap

clean up, create NER output

use wordlist to duplicate sentences to supplement training data

In [1]:
from collections import Counter
from nltk import pos_tag, word_tokenize
from dataset import get_vocab, index_sents
import pandas as pd
import numpy as np
import re

In [2]:
# data = pd.read_csv('data/merged_corpus_phrases_new_fixed.csv')
data = pd.read_csv('../00_data/merged_corpus.csv')

In [3]:
# cast to str due to number-only lines
sents = [str(s) for s in data['message'].tolist()]
speechacts = data['speech_act'].tolist()
topics = [str(s) for s in data['topic'].tolist()]

## tokenize

In [4]:
# fix 'p m'
fixed = []
for idx, sent in enumerate(sents):
    sent = sent.replace(' p m', ' pm')
    sent = sent.replace(' p m.', ' pm.')
    sent = sent.replace(' p m?', ' pm?')
    sent = sent.replace(' p m,', ' pm,')
    sent = sent.replace(' p m ', ' pm ')
    sent = sent.replace(' a m.', ' am.')
    sent = sent.replace(' a m?', ' am?')
    sent = sent.replace(' a m,', ' am,')
    sent = sent.replace(' a m ', ' am ')
    sent = re.sub('a\sm$', 'am', sent)
    fixed.append(sent)

In [5]:
# tokenize and pos-tag with NLTK
postagged = [pos_tag(word_tokenize(s)) for s in fixed]

In [6]:
lens = [len(l) for l in postagged]
max(lens), (sum(lens)/len(lens))

(49, 4.349464036041634)

In [7]:
postagged[0]

[('good', 'JJ'), ('afternoon', 'NN')]

In [8]:
names = []
for sent in fixed:
    words = re.findall(r'[A-Z][a-z]+', sent)
    for word in words:
        names.append(word)

nameCounts = Counter(names)
nameCounts.most_common(10) # remove 10 to see all

[('San', 183),
 ('Francisco', 114),
 ('United', 106),
 ('Euston', 95),
 ('American', 95),
 ('Saturday', 75),
 ('Virgin', 73),
 ('October', 73),
 ('Monday', 67),
 ('Sunday', 60)]

In [9]:
names = []
for sent in fixed:
    words = re.findall(r'[A-Z][A-Z]+', sent)
    for word in words:
        names.append(word)

nameCounts = Counter(names)
nameCounts.most_common(10) # remove 10 to see all

[('US', 39),
 ('WWW', 27),
 ('LA', 12),
 ('SFO', 9),
 ('TWA', 9),
 ('LAX', 8),
 ('JFK', 8),
 ('DDD', 7),
 ('CC', 6),
 ('CCC', 6)]

## NER auto-tagging

this code is messy

In [10]:
# define lists
timewords = ['midnight', 'noon', 'am', 'pm', 'morning', 'afternoon', 'evening', 'night']

datewords = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday', 'January', 'February',
             'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December',
             'tomorrow']

# from transdict finds
placewords = ['Euston', 'London', 'Birmingham', 'Street', 'Chicago', 'Wilmslow', 'Macclesfield', 'Stockport',
              'Piedmont', 'Dallas', 'Newark', 'Wigan', 'Preston', 'Denver', 'Liverpool', 'Seattle', 'Tokyo',
              'Wrexham', 'Richmond' , 'Manchester', 'Crewe', 'Baltimore', 'Ottawa', 'Toronto', 'Vancouver', 
              'Moscow', 'Boston', 'Edinburgh', 'Oakland', 'Newcastle', 'Durham', 'Lime', 'Taunton', 
              'Copenhagen', 'Heathrow', 'Helsinki', 'Pittsburg', 'Raleigh', 'Picadilly', 'Watford', 'Hertford',
              'Leicester', 'Newton', 'Abbot', 'Greenbay', 'Miami', 'Orlando', 'Washington', 'Dulles', 'Charlotte',
              'Tahoe', 'Southbend', 'Springfield', 'York', 'Burbank', 'Syracuse', 'Cleveland', 'Fairbanks',
              'Carolina', 'Montreal', 'Wolverhampton', 'Leeds', 'Derby', 'Blackpool', 'Oxenholme', 'Ontario',
              'Riyadh', 'Portland', 'Barclay', 'Calgary', 'Bangkok', 'Burigan', 'Nantucket', 'Menlo', 'Cottage',
              'Wisconsin', 'Gatwick', 'Singapore', 'Irvine', 'Frankfurt', 'Jersey', 'Columbus', 'Merseyside', 
              'Fanshawe', 'Essex', 'Stafford', 'Philadelphia', 'Switzerland', 'Denmark', 'Sandestrom',
              'Braniff', 'Stockholm', 'Sweden', 'Germany', 'Belgium', 'Brussels', 'Rochester', 'Anchorage', 
              'California', 'Arkansas', 'England', 'Michigan', 'Detroit', 'Indiana', 'Louisville', 'Ohio',
              'Tulsa', 'Indianapolis', 'Milwaukee', 'Oklahoma', 'Colorado', 'Virginia', 'Coxhoe', 'Redwich',
              'Camden', 'Leicestershire', 'Cumbria', 'Heswall', 'Wirral', 'Liver', 'Cheshire', 'Goldhampton',
              'International', 'Airport', 'Central', 'General', 'St', 'Atlanta', 'Nuneaton', 'Beijing', 'Europe',
              'LA', 'ORD', 'SFO', 'JFK', 'LAX', 'SAS', 'SF', 'Pancreas', 'Rugby']

# from transdict finds
companywords = ['United', 'American', 'Virgin', 'Express', 'Delta', 'Trainlines', 'Visa', 'Airlines', 'Travel',
                'Hertz', 'Lufthansa', 'Northwest', 'Canadian', 'British', 'Trainline', 'Northwestern',
                'Mastercard', 'Airline', 'Southwestern', 'Telesales', 'Merchandising', 'Aeroflight',
                'Airways', 'Korean', 'Hotel', 'Alaskan', 'Alaska', 'TWA', 'Eagle', 'Saudia', 'Arabian', 
                'PanAm', 'CP', 'Air', 'trainlines', 'Continental']

# seat-class words
ticketwords = ['Coach', 'Business', 'non-stop', 'Round-trip', 'Advance',
               'Super', 'Value', 'Return', 'Saver', 'First', 'Class', 'Single']

# names
namewords = ['Sandra']

# if number + these = time
timeparts = ["o'clock"]

# split names
placefirsts = ['Palo', 'Green', 'New', 'Lime', 'Orange', 'San', 'Los', 'Hong', 'John', 'Las', 'Saudi', 'Saint',
               'Santa', 'Soviet', 'Little']
placelasts = ['Alto', 'Bay', 'Lime' 'Street', 'County', 'Francisco', 'Angeles', 'Diego', 'Jose', 'Bernadino',
              'Kong', 'Wayne', 'Vegas', 'Arabia', 'Louis', 'Petersburg', 'Ana', 'Union', 'Rock']

compfirsts = ['Cathay', 'Pan']
complasts = ['Pacific', 'Am']

currencies = ['pounds', 'dollars', 'cents', 'dollar']

numbers = ['hundred', 'hundreds', 'thousand', 'thousands']

In [11]:
timetag = 'TIM'
datetag = 'DAT'
citytag = 'GEO'
comptag = 'COM'
classtag = 'TIX'
flighttag = 'FLT'
currtag = 'CUR'
numtag = 'NUM'

# to save
alltoks = []
alltags = []
allners = []

# to print
makecsv = []

indices = []

# iterate
for jdx, seq in enumerate(postagged):
    
    # initialize
    toks = [t[0] for t in seq]
    tags = [t[1] for t in seq]
    ners = ['O' for i in range(len(toks))]

    # iterate and list-replace
    for idx, tok in enumerate(toks[:-1]):
        # time words using list substitution
        if tok in timewords:
            ners[idx] = timetag
        # date words
        elif tok in datewords:
            ners[idx] = datetag
        # date words
        elif tok in currencies:
            ners[idx] = currtag
        # place names using list substitution
        elif tok in placewords:
            ners[idx] = citytag
        # company names using list substitution
        elif tok in companywords:
            ners[idx] = comptag
        # ticket class words
        elif tok in ticketwords:
            ners[idx] = classtag
        
        # o'clock time using regex
        if re.match(r'[0-9]+', tok):
            if toks[idx+1] in timeparts:
                ners[idx] = timetag
                ners[idx+1] = timetag
         
        # try flight numbers
        if tok == 'flight' and re.findall(r'[0-9]', toks[idx+1]):
            ners[idx] = flighttag
        
        # time re search
        if re.findall(r'[0-9]+\:[0-9]+(\.\,\?)*', tok):
            ners[idx] = timetag    
        
        # else assume flight number unless before prep
        if ners[idx] == 'O' and re.match(r'[0-9]+', tok):
            if toks[idx+1] in ['hours','minutes','miles','kilometers', 'person', 'persons', 'people', 'members']:
                ners[idx] = numtag
            elif toks[idx+1] in currencies:
                ners[idx] = currtag
            elif toks[idx+1] in timewords or toks[idx+1] in timeparts:
                ners[idx] = timetag
            elif idx > 0:
                if toks[idx-1] in currencies:
                    ners[idx] = currtag
                elif ners[idx-1] == flighttag or ners[idx-1] == comptag:
                    ners[idx] = flighttag
                else:
                    ners[idx] = numtag
            else:
                ners[idx] = numtag
        
        # ordinal numbers for dates
        if re.findall(r'[0-9]+(nd|st|th|rd)', tok):
            ners[idx] = datetag
                
        # two-word place names
        if tok in placefirsts:
            if toks[idx+1] in placelasts:
                ners[idx] = citytag
                ners[idx+1] = citytag
                
        # two-word company names
        if tok in compfirsts:
            if toks[idx+1] in complasts:
                ners[idx] = comptag
                ners[idx+1] = comptag
                
        # ticket class words
        elif tok in timeparts:
            ners[idx-1] = timetag
            ners[idx] = timetag
                
    # last word
    # time words using list substitution
    if toks[-1] in timewords:
        ners[-1] = timetag
    # date words
    elif toks[-1] in datewords:
        ners[-1] = datetag
    elif toks[-1] in numbers:
        ners[-1] = numtag
    elif toks[-1] in currencies:
        ners[-1] = currtag
    # place names using list substitution
    elif toks[-1] in placewords:
        ners[-1] = citytag
    # company names using list substitution
    elif toks[-1] in companywords:
        ners[-1] = comptag
    # ticket class words
    elif toks[-1] in ticketwords:
        ners[-1] = classtag
    # time re search
    if re.match(r'[0-9]+\:[0-9]+(\.\,\?)*', toks[-1]):
        ners[-1] = timetag 
    # ordinal numbers for dates
    elif re.findall(r'[0-9]+(nd|st|th|rd)', toks[-1]):
        ners[-1] = datetag
    elif len(seq) > 1 and re.match(r'[0-9]+', toks[-1]): 
        if ners[-2] == flighttag or ners[-2] == numtag:
            ners[-1] = ners[-2]
        elif toks[-1] in currencies:
            ners[-1] = currtag
        elif tags[-2] == 'IN':
            ners[-1] = timetag
        else:
            ners[-1] = numtag
    
    # final pass
    for i in range(len(toks)):
        if i < len(toks)-1:
            if toks[i] == 'good' and toks[i+1] == 'morning' or toks[i+1] == 'afternoon':
                ners[i+1] = 'O'        
    
    answ = pd.DataFrame(
    {
        'toks': toks,
        'tags': tags,
        'ners': ners,
        'skip' : [' ' for s in toks]
    })

    answ = answ[['toks', 'ners', 'skip']]
    answ = answ.T
    
    # add to test data
    alltoks.append(toks)
    alltags.append(tags)
    allners.append(ners)
    indices.append(jdx)  # save sentence index
    makecsv.append(answ) # append df to list seq]

In [12]:
vocab = [w for s in alltoks for w in s]
vcounts = Counter(vocab)
vcounts.most_common(10)

[('the', 2127),
 ('to', 1592),
 ('you', 1487),
 ("'s", 1440),
 ('i', 1364),
 ('that', 1314),
 ('and', 1304),
 ('it', 1062),
 ('ok', 1008),
 ('a', 913)]

In [13]:
df = pd.concat(makecsv)

In [14]:
len([x for x in postagged if len(x)> 2])

6677

## NER training prototype data

In [15]:
len(speechacts), len(alltoks), len(alltags), len(allners)

(12874, 12874, 12874, 12874)

In [16]:
saCounter = Counter(speechacts)
len(set(speechacts))

262

In [17]:
# remove the speech-act prefixes
truncsa = [sa.split('-')[-1] for sa in speechacts]
tsaCounter = Counter(truncsa)
len(set(truncsa)), tsaCounter.most_common()

(65,
 [('state', 2267),
  ('acknowledge', 2186),
  ('refer', 1706),
  ('reqInfo', 1060),
  ('init', 999),
  ('hesitate', 518),
  ('reqConfirm', 411),
  ('abandon', 372),
  ('hold', 293),
  ('negate', 188),
  ('thank', 186),
  ('stateIntent', 181),
  ('exclaim', 179),
  ('stateCondition', 179),
  ('stateConstraint', 173),
  ('approve', 170),
  ('bye', 166),
  ('predict', 163),
  ('expressOpinion', 149),
  ('reqDirect', 146),
  ('agree', 126),
  ('expressPossibility', 112),
  ('identifySelf', 104),
  ('greet', 96),
  ('suggest', 76),
  ('direct', 64),
  ('stateReason', 62),
  ('apologise', 57),
  ('expressWish', 40),
  ('pardon', 36),
  ('stateOpt', 33),
  ('reqOpt', 32),
  ('confirm', 26),
  ('uninterpretable', 26),
  ('expressNonAwareness', 25),
  ('expressImPossibility', 24),
  ('echo', 24),
  ('correct', 22),
  ('accept', 20),
  ('phatic', 19),
  ('correctSelf', 19),
  ('offer', 18),
  ('expressRegret', 16),
  ('expressAwareness', 16),
  ('complete', 12),
  ('reject', 12),
  ('elab',

In [18]:
# remove the topic prefixes, suffixes
tpCounter = Counter(topics)
trunctop = [t.split('-')[-1] for t in topics]
trunctop = [t.split('_')[0] for t in trunctop]
trunctopCounter = Counter(trunctop)
len(set(topics)), len(set(trunctop)), trunctopCounter.most_common()

(705,
 40,
 [('none', 7868),
  ('enum', 1201),
  ('time', 487),
  ('day', 457),
  ('location', 395),
  ('number', 332),
  ('arrival', 239),
  ('journey', 188),
  ('airline', 176),
  ('creditcard', 164),
  ('date', 152),
  ('from', 137),
  ('name', 123),
  ('verify', 104),
  ('seat', 104),
  ('fare', 90),
  ('departure', 84),
  ('address', 79),
  ('availability', 68),
  ('month', 58),
  ('railcard', 53),
  ('cancel', 49),
  ('booking', 48),
  ('return', 42),
  ('district', 32),
  ('to', 30),
  ('confirm', 22),
  ('problem', 17),
  ('miss', 14),
  ('airport', 13),
  ('hotel', 12),
  ('nan', 9),
  ('refund', 7),
  ('duration', 6),
  ('car', 6),
  ('week', 3),
  ('country', 2),
  ('spell', 1),
  ('disflu', 1),
  ('penalty', 1)])

In [19]:
topicSA = [truncsa[i]+'-'+trunctop[i] for i in range(len(topics))]
topicSACounter = Counter(topicSA)
len(set(topicSA)) #, topicSACounter.most_common()

503

In [20]:
# filter by length, remove all of length 2 or under
indices = []
for idx, tokseq in enumerate(alltoks):
    if len(tokseq) > 2:
        indices.append(idx)

In [21]:
shortTops = [topicSA[i] for i in indices]
shortTopCounter = Counter(shortTops)

In [22]:
short_speechacts = [truncsa[i] for i in indices]
short_topics = [trunctop[i] for i in indices]
short_topic_acts = shortTops[:]
short_tokens = [alltoks[i] for i in indices]
short_postags = [alltags[i] for i in indices]
short_nertags = [allners[i] for i in indices]

## supplement data

In [23]:
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

months = ['January', 'February', 'March', 'April', 'May', 'June', 
          'July', 'August', 'September', 'October', 'November', 'December']

cities = ['Chicago', 'Wilmslow', 'Macclesfield', 'Stockport',
          'Piedmont', 'Dallas', 'Newark', 'Wigan', 'Preston', 'Denver', 'Liverpool', 'Seattle', 'Tokyo',
          'Wrexham', 'Richmond' , 'Manchester', 'Crewe', 'Baltimore', 'Ottawa', 'Toronto', 'Vancouver', 
          'Moscow', 'Boston', 'Edinburgh', 'Oakland', 'Newcastle', 'Durham', 'Taunton', 
          'Copenhagen', 'Helsinki', 'Pittsburg', 'Raleigh', 'Picadilly', 'Watford',
          'Leicester', 'Newton', 'Abbot', 'Greenbay', 'Miami', 'Orlando', 'Charlotte',
          'Tahoe', 'Southbend', 'Springfield', 'Burbank', 'Syracuse', 'Cleveland', 'Fairbanks',
          'Montreal', 'Wolverhampton', 'Leeds', 'Derby', 'Blackpool', 'Oxenholme', 'Ontario',
          'Riyadh', 'Portland', 'Barclay', 'Calgary', 'Bangkok', 'Burigan', 'Nantucket', 'Menlo', 'Cottage',
          'Gatwick', 'Singapore', 'Irvine', 'Frankfurt', 'Jersey', 'Columbus', 'Merseyside', 
          'Fanshawe', 'Essex', 'Stafford', 'Philadelphia', 'Sandestrom',
          'Braniff', 'Stockholm', 'Brussels', 'Rochester', 'Anchorage', 
          'Detroit', 'Indiana', 'Louisville',
          'Tulsa', 'Indianapolis', 'Milwaukee', 'Oklahoma', 'Coxhoe', 'Redwich',
          'Camden', 'Leicestershire', 'Cumbria', 'Heswall', 'Wirral', 'Liver', 'Goldhampton',
          'Atlanta', 'Nuneaton', 'Beijing']

newcities = ['Shanghai', 'Seoul', 'Busan', 'Daegu', 'Paris', 'Istanbul', 'Delhi', 'Mumbai', 'Cairo',
             'Tehran', 'Ankara', 'Berlin', 'Madrid', 'Incheon', 'Rome', 'Osaka',
             'Nagoya', 'Manila', 'Minsk', 'Budapest', "Warsaw", "Phoenix"]

airlines = ['United', 'Virgin', 'Delta', 'Trainlines',
            'Hertz', 'Lufthansa', 'Northwest', 'Canadian', 'British', 'Northwestern',
            'Airline', 'Southwestern', 'Aeroflight', 'Continental',
            'Airways', 'Korean', 'Alaskan', 'Alaska', 'TWA', 'Eagle', 'Saudia', 'Arabian', 
            'PanAm', 'CP', 'Air']

newairlines = ['Allegiant', 'Frontier', 'JetBlue', 'Spirit', 'Envoy', 'SkyWest', 'Quantas',
               'Etihad', 'Dragonair', 'AirAsia', 'Finnair', 'Aegean', 'easyJet', 'WestJet',
               'Indigo', 'Iberia', 'Emirates', 'EVA', 'ANAs', 'JAL']

journey = ['journey', 'trip', 'itinerary']

traveling = ['travelling', 'traveling', 'flying']

depart = ['depart', 'leave']

departing = ['departing', 'leaving']

book = ['book', 'schedule', 'reserve']

In [24]:
add_speechacts = []
add_topics = []
add_topic_acts = []
add_tokens = []
add_postags = []
add_nertags = []

for i in range(len(short_speechacts)):
    
    iters = np.random.randint(4, 8)
    for j in range(iters):
    
        toks = short_tokens[i]
        tags = short_postags[i]
        ners = short_nertags[i]

        new_toks = toks[:]

        # check each list
        for lst in [days, months, cities, airlines, journey, traveling, depart, departing, book]:

            # check each word
            for idx, tok in enumerate(toks):

                # if found, randomly sample new word
                if tok in lst:
                    new = tok
                    while new == tok:
                        if lst == cities:
                            new = newcities[np.random.randint(0, len(newcities))]
                        elif lst == airlines:
                            new = newairlines[np.random.randint(0, len(newairlines))]
                        else:
                            new = lst[np.random.randint(0, len(lst))]
                    # replace
                    new_toks[idx] = new
        
        # update if new sent
        if toks != new_toks:
            add_tokens.append(new_toks)
            add_postags.append(short_postags[i])
            add_nertags.append(short_nertags[i])
            add_speechacts.append(short_speechacts[i])
            add_topics.append(short_topics[i])
            add_topic_acts.append(short_topic_acts[i])
    
    if i % 1000 == 0:
        print("done", i, "of", len(short_speechacts))
        
len(add_tokens), len(add_postags), len(add_nertags), len(add_speechacts), len(add_topics), len(add_topic_acts)

done 0 of 6677
done 1000 of 6677
done 2000 of 6677
done 3000 of 6677
done 4000 of 6677
done 5000 of 6677
done 6000 of 6677


(8901, 8901, 8901, 8901, 8901, 8901)

In [25]:
for i in range(100,105):
    print(add_tokens[i])
    print(add_postags[i])
    print(add_nertags[i])
    print('')

['could', 'i', 'have', 'front', 'facing', 'from', 'Cairo', 'to', 'erm']
['MD', 'VB', 'VBP', 'VBN', 'VBG', 'IN', 'NNP', 'TO', 'VB']
['O', 'O', 'O', 'O', 'O', 'O', 'GEO', 'O', 'O']

['Tuesday', 'the', '24th', 'of', 'April']
['NNP', 'DT', 'CD', 'IN', 'NNP']
['DAT', 'O', 'DAT', 'O', 'DAT']

['Monday', 'the', '24th', 'of', 'September']
['NNP', 'DT', 'CD', 'IN', 'NNP']
['DAT', 'O', 'DAT', 'O', 'DAT']

['Friday', 'the', '24th', 'of', 'August']
['NNP', 'DT', 'CD', 'IN', 'NNP']
['DAT', 'O', 'DAT', 'O', 'DAT']

['Sunday', 'the', '24th', 'of', 'November']
['NNP', 'DT', 'CD', 'IN', 'NNP']
['DAT', 'O', 'DAT', 'O', 'DAT']



In [26]:
add_speechacts += short_speechacts
add_topics += short_topics
add_topic_acts += short_topic_acts
add_tokens += short_tokens
add_postags += short_postags
add_nertags += short_nertags

In [27]:
def lowercase(lol):
    result = []
    for lst in lol:
        this_sent = []
        for w in lst:
            for number in ['1','2','3','4','5','6','7','8','9','0']:
                w = w.replace(number, '#')
            this_sent.append(w.lower())
        result.append(this_sent)
    return result

add_tokens = lowercase(add_tokens)

In [28]:
# shuffle data
np.random.seed(1337)
sindices = list(np.random.permutation(len(add_speechacts)))

add_speechacts = [add_speechacts[i] for i in sindices]
add_topics = [add_topics[i] for i in sindices]
add_topic_acts = [add_topic_acts[i] for i in sindices]
add_tokens = [add_tokens[i] for i in sindices]
add_postags = [add_postags[i] for i in sindices]
add_nertags = [add_nertags[i] for i in sindices]

In [29]:
sindices[:10]

[1656, 11389, 14324, 13129, 11669, 928, 6986, 3248, 11015, 14897]

## create vocab and index dicts

In [30]:
word2idx, idx2word = get_vocab(add_tokens, 10000)
pos2idx, idx2pos = get_vocab(alltags, len(set([tag for sent in add_postags for tag in sent]))+2)
ner2idx, idx2ner = get_vocab(allners, len(set([tag for sent in add_nertags for tag in sent]))+2)

In [31]:
np.save('../00_data/encoded/word2idx.npy', word2idx)
np.save('../00_data/encoded/idx2word.npy', idx2word)
np.save('../00_data/encoded/pos2idx.npy', pos2idx)
np.save('../00_data/encoded/idx2pos.npy', idx2pos)
np.save('../00_data/encoded/ner2idx.npy', ner2idx)
np.save('../00_data/encoded/idx2ner.npy', idx2ner)

In [32]:
np.save('../00_data/encoded/add_speechacts.npy', add_speechacts)
np.save('../00_data/encoded/add_topics.npy', add_topics)
np.save('../00_data/encoded/add_topic_acts.npy', add_topic_acts)
np.save('../00_data/encoded/add_tokens.npy', add_tokens)
np.save('../00_data/encoded/add_postags.npy', add_postags)
np.save('../00_data/encoded/add_nertags.npy', add_nertags)